## Kaggle + WLASL setup (patched)
This notebook is patched to run in **Kaggle** using **/kaggle/input/wlasl-processed** and a local subset JSON.

**You must add your subset JSON** (e.g., `wlasl_subset.json`) to the notebook. Easiest options:
- Upload it as a Kaggle Dataset and point `SUBSET_JSON` to that path, **or**
- Upload it during the session and place it at `/kaggle/working/wlasl_subset.json`.


In [ ]:
import os, glob, json

# Kaggle paths
INPUT_DIR = "/kaggle/input/wlasl-processed"
BASE_DIR  = "/kaggle/working/sign_language_detection"
os.makedirs(BASE_DIR, exist_ok=True)

# Put your filtered subset JSON here.
# Option A: if you upload it to the notebook session, keep it in /kaggle/working
# use SUBSET_JSON = "/kaggle/input/wlasl-subset for a tinier set"
SUBSET_JSON = "/kaggle/input/wlasl-subset"

# Option B: if you attached it as a Kaggle dataset, set something like:
# SUBSET_JSON = "/kaggle/input/<your-dataset-name>/wlasl_subset.json"

assert os.path.exists(INPUT_DIR), f"Missing INPUT_DIR: {INPUT_DIR}"
assert os.path.exists(SUBSET_JSON), f"Missing SUBSET_JSON: {SUBSET_JSON} (upload it or attach as dataset)"

DATA_PATH = os.path.join(BASE_DIR, "MP_Data")
os.makedirs(DATA_PATH, exist_ok=True)

print("INPUT_DIR:", INPUT_DIR)
print("SUBSET_JSON:", SUBSET_JSON)
print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_PATH)


In [ ]:
# Backwards-compat alias used by the original notebook
base_dir = BASE_DIR
base_dir


# Installing Dependencies

In [ ]:
!pip3 uninstall -y mediapipe
!pip3 install -U "numpy<2" "protobuf<5"
!pip3 install --no-cache-dir "mediapipe==0.10.21"

In [ ]:
# This command installs the TensorFlow library using pip (Python's package installer)
!pip3 install tensorflow

In [ ]:
# Install the OpenCV library, which is used for computer vision tasks such as image and video processing.
!pip install opencv-python

# 1. Importing Libraries

In [ ]:
# General libraries
import os  # For interacting with the operating system (e.g., file handling)
import time  # For time-related functions
import numpy as np  # For numerical operations and handling arrays
import random
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt  # For plotting graphs and images
from IPython.display import display, Image  # For displaying images and other media in Jupyter/Colab

# Computer Vision Libraries
import cv2  # For computer vision tasks such as image and video processing
import mediapipe as mp  # For using MediaPipe's machine learning models for pose detection and more



# Deep Learning Libraries (TensorFlow and Keras)
from tensorflow.keras.utils import to_categorical  # For converting labels to categorical format (one-hot encoding)
from tensorflow.keras.models import Sequential  # For defining the neural network architecture
from tensorflow.keras.layers import LSTM, Dense  # For adding LSTM and Dense layers to the model
from tensorflow.keras.callbacks import TensorBoard  # For logging training progress for TensorBoard
from tensorflow.keras.models import load_model  # For loading pre-trained models

In [ ]:
test_videos_path = os.path.join(BASE_DIR, 'test_videos')
os.makedirs(test_videos_path, exist_ok=True)
print('test_videos_path:', test_videos_path)


# 2. Detecting Keypoints using MP Holistic

Code Block 2A

In [ ]:
# Initialize the Holistic model from MediaPipe for full-body pose detection.
# This model can detect landmarks for the face, hands, and body.
mp_holistic = mp.solutions.holistic

# Initialize the drawing utilities from MediaPipe to draw the landmarks on images or videos.
# These utilities help visualize the pose detection results by drawing the keypoints and connections.
mp_drawing = mp.solutions.drawing_utils

# This function processes an image using a MediaPipe model to detect landmarks/poses and returns the processed image and the results.
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# This function draws the detected pose, left hand, and right hand landmarks with their connections on the given image.
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

# This function draws the pose, left hand, and right hand landmarks with custom styling (color, thickness, and circle radius)
# on the given image, using the MediaPipe Holistic model's results.
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

# 3. Extracting Keypoint Values

Code Block 3A

In [ ]:


# This function extracts keypoints (pose, face, left hand, right hand) from the results of MediaPipe landmarks,
# flattens the data into 1D arrays, and concatenates them into a single array for further processing.

def extract_keypoints(results):
    pose = np.array([[r.x, r.y, r.z, r.visibility] for r in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh   = np.array([[r.x, r.y, r.z] for r in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh   = np.array([[r.x, r.y, r.z] for r in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh]).astype(np.float32)


In [ ]:
# Video Download Utilities
import urllib.request
import hashlib

DOWNLOAD_CACHE_DIR = "/kaggle/working/downloaded_videos"
os.makedirs(DOWNLOAD_CACHE_DIR, exist_ok=True)

def is_url(path_or_url: str) -> bool:
    """Check if a string is a URL"""
    return path_or_url.startswith(('http://', 'https://'))

def download_video(url: str, cache_dir: str = DOWNLOAD_CACHE_DIR, timeout: int = 30) -> str:
    """Download a video from URL and cache it locally"""
    # Create a hash of the URL for the filename
    url_hash = hashlib.md5(url.encode()).hexdigest()
    cached_path = os.path.join(cache_dir, f"{url_hash}.mp4")
    
    # Return cached file if it exists
    if os.path.exists(cached_path):
        return cached_path
    
    # Download the video
    try:
        print(f"  📥 Downloading: {url[:60]}...")
        urllib.request.urlretrieve(url, cached_path)
        print(f"  ✅ Downloaded to: {cached_path}")
        return cached_path
    except Exception as e:
        print(f"  ❌ Failed to download {url}: {e}")
        return None

print("✅ Video download utilities ready")


In [ ]:
# Enhanced find_video_path with URL download support
# This extends the existing find_video_path function

def find_video_path_enhanced(video_id: str, instance_data: dict = None):
    """
    Enhanced version that also checks for URLs in instance data.
    Use this instead of find_video_path if you want URL downloading.
    """
    # First try the original function
    result = find_video_path(video_id)
    if result:
        return result
    
    # Check if video_id itself is a URL
    if is_url(video_id):
        downloaded = download_video(video_id)
        if downloaded:
            video_path_cache[video_id] = downloaded
            return downloaded
    
    # Check instance data for URL field
    if instance_data and "url" in instance_data:
        url = instance_data["url"]
        if is_url(url):
            downloaded = download_video(url)
            if downloaded:
                video_path_cache[video_id] = downloaded
                return downloaded
    
    return None

# Replace the original function (comment out if you want to keep both)
find_video_path_original = find_video_path  # Keep backup
find_video_path = find_video_path_enhanced  # Use enhanced version

print("✅ Enhanced find_video_path with URL support ready")


# 4. WLASL subset: build dataset (videos → MediaPipe keypoints → X/y)
This replaces the manual folder-collection workflow. It will:
- read your **filtered WLASL JSON**
- find matching `.mp4` files under `/kaggle/input/wlasl-processed`
- extract **30 frames** per clip with MediaPipe Holistic
- build `X_train, y_train, X_val, y_val, X_test, y_test` using the JSON `split` field


In [ ]:
with open("/kaggle/input/wlasl-subset/wlasl_subset.json") as f:
    data = json.load(f)

actions = sorted({item["gloss"] for item in data})
actions = np.array(actions)

print("Actions:", actions)


In [ ]:
# TODO: Define the set of actions (sign language gestures) to detect

import os, shutil

DATA_PATH = os.path.join(BASE_DIR, "MP_Data")

if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH)

os.makedirs(DATA_PATH, exist_ok=True)

print("✅ Deleted and recreated:", DATA_PATH)



actions = np.array(actions)

# Path for exported data (numpy arrays will be saved here)
DATA_PATH = os.path.join(BASE_DIR, 'MP_Data')
VIDEO_PATH = os.path.join(BASE_DIR, 'videos')

# Create the data directory if it doesn't exist
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Number of sequences (videos) to record per action
no_sequences = 30

# Number of frames per video sequence
sequence_length = 30

# Starting index for naming folders that store each sequence
start_folder = 30

# Create folder structure for each action and each sequence
for action in actions:
    # Path for this action's data
    action_path = os.path.join(DATA_PATH, action)
    video_path = os.path.join(VIDEO_PATH, action)

    print(action_path)

    # Create action folder if it doesn't exist
    if not os.path.exists(action_path):
        os.makedirs(action_path)

    # Create video folder if it doesn't exist
    if not os.path.exists(video_path):
        os.makedirs(video_path)

    # Find existing numbered folders (sequences) for this action
    existing_dirs = [d for d in os.listdir(action_path) if d.isdigit()]

    # Determine the current max sequence number to avoid overwriting
    if existing_dirs:
        dirmax = np.max(np.array(existing_dirs).astype(int))
    else:
        dirmax = 0

    # Create subfolders for each new sequence
    for sequence in range(0, no_sequences):
        seq_path = os.path.join(action_path, str(dirmax + sequence))
        if not os.path.exists(seq_path):
            os.makedirs(seq_path)

In [ ]:
import os, json, glob
import numpy as np
import cv2
from tqdm import tqdm
from collections import defaultdict

WLASL_ROOT = "/kaggle/input/wlasl-processed"
VIDEOS_DIR = os.path.join(WLASL_ROOT, "videos")
DATA_PATH = "/kaggle/working/sign_language_detection/MP_Data"

SEQUENCE_LENGTH = 30
MAX_PER_CLASS = None  # e.g. 50 for testing

# Load your subset JSON (already in `data` per your notebook)
# data = json.load(open("wlasl_subset.json"))

print("🔍 Building video path index...")
all_mp4 = glob.glob(os.path.join(WLASL_ROOT, "**/*.mp4"), recursive=True)
# Create a faster lookup: video_id -> path
video_path_cache = {}
for p in all_mp4:
    basename = os.path.basename(p)
    # Try exact match first
    vid_id = basename.replace(".mp4", "")
    video_path_cache[vid_id] = p
    # Also index by partial matches
    if "_" in vid_id:
        parts = vid_id.split("_")
        for part in parts:
            if part and part not in video_path_cache:
                video_path_cache[part] = p

def find_video_path(video_id: str):
    # Try exact match first
    if video_id in video_path_cache:
        return video_path_cache[video_id]
    # Try with .mp4 extension
    direct = os.path.join(VIDEOS_DIR, f"{video_id}.mp4")
    if os.path.exists(direct):
        video_path_cache[video_id] = direct
        print("video exists!")
        return direct
    # Try partial matches
    for cached_id, path in video_path_cache.items():
        if video_id in cached_id or cached_id in video_id:
            return path
    return None

def clamp_bbox(bbox, w, h):
    x1, y1, x2, y2 = bbox
    x1 = int(max(0, min(w-1, x1)))
    x2 = int(max(0, min(w-1, x2)))
    y1 = int(max(0, min(h-1, y1)))
    y2 = int(max(0, min(h-1, y2)))
    if x2 <= x1 or y2 <= y1:
        return None
    return x1, y1, x2, y2

def sample_indices(start, end, n=30):
    # inclusive start/end frames
    if end < start:
        return None
    total = end - start + 1
    if total <= 0:
        return None
    # Use uniform sampling
    rel = np.linspace(0, total - 1, n).astype(int)
    return (start + rel).tolist()

# Build per-gloss list of instances
by_gloss = {}
for entry in data:
    gloss = entry.get("gloss")
    insts = entry.get("instances", [])
    if not gloss or not isinstance(insts, list):
        continue
    by_gloss.setdefault(gloss, [])
    for inst in insts:
        vid = inst.get("video_id")
        if vid is None:
            continue
        by_gloss[gloss].append(inst)
        
total_instances = sum(len(v) for v in by_gloss.values())
print("Total instances to process:", total_instances)

# optionally cap per class
if MAX_PER_CLASS is not None:
    for g in by_gloss:
        by_gloss[g] = by_gloss[g][:MAX_PER_CLASS]

os.makedirs(DATA_PATH, exist_ok=True)

# Track skip reasons for debugging
skip_reasons = defaultdict(int)

def process_video(inst, gloss, seq_id, holistic):
    """Process a single video instance. Returns (success, skip_reason)"""
    video_id = str(inst["video_id"])
    video_path = find_video_path(video_id)
    
    if video_path is None:
        return False, "video_not_found"
    
    # Check if already processed
    gloss_dir = os.path.join(DATA_PATH, gloss)
    out_dir = os.path.join(gloss_dir, str(seq_id))
    if os.path.exists(out_dir):
        npy_files = glob.glob(os.path.join(out_dir, "*.npy"))
        if len(npy_files) == SEQUENCE_LENGTH:
            return True, None  # Already processed successfully
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return False, "cannot_open"
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 25.0
    
    if total_frames <= 0:
        cap.release()
        return False, "zero_frames"
    
    # Handle frame_start and frame_end
    fs = inst.get("frame_start", 0)
    fe = inst.get("frame_end", -1)
    
    # If frame_end is -1 or None, use the full video
    if fe == -1 or fe is None:
        fe = total_frames - 1
    
    # Ensure valid frame range
    fs = max(0, min(total_frames - 1, int(fs)))
    fe = max(0, min(total_frames - 1, int(fe)))
    
    # Ensure we have enough frames
    if fe - fs + 1 < SEQUENCE_LENGTH:
        # Try to expand the range if possible
        needed = SEQUENCE_LENGTH - (fe - fs + 1)
        expand_left = min(needed // 2, fs)
        expand_right = min(needed - expand_left, total_frames - 1 - fe)
        fs = max(0, fs - expand_left)
        fe = min(total_frames - 1, fe + expand_right)
    
    if fe < fs or (fe - fs + 1) < SEQUENCE_LENGTH:
        cap.release()
        return False, "insufficient_frames"
    
    idxs = sample_indices(fs, fe, SEQUENCE_LENGTH)
    if idxs is None or len(idxs) < SEQUENCE_LENGTH:
        cap.release()
        return False, "invalid_indices"
    
    idxs_set = set(idxs)
    os.makedirs(out_dir, exist_ok=True)
    
    bbox = inst.get("bbox", None)
    saved = 0
    current = 0
    
    # Seek to start frame for faster processing
    cap.set(cv2.CAP_PROP_POS_FRAMES, fs)
    
    while cap.isOpened() and saved < SEQUENCE_LENGTH:
        ret, frame = cap.read()
        if not ret or frame is None:
            break
        
        # Check if we've gone past the end frame
        if current + fs > fe:
            break
        
        if (current + fs) in idxs_set:
            # Apply bbox if available
            if bbox is not None and isinstance(bbox, (list, tuple)) and len(bbox) == 4:
                h, w = frame.shape[:2]
                bb = clamp_bbox(bbox, w, h)
                if bb is not None:
                    x1, y1, x2, y2 = bb
                    # Ensure bbox is valid
                    if x2 > x1 and y2 > y1:
                        frame = frame[y1:y2, x1:x2]
                    else:
                        # Invalid bbox, skip this frame
                        current += 1
                        continue
            
            # Process with MediaPipe
            try:
                _, results = mediapipe_detection(frame, holistic)
                keypoints = extract_keypoints(results) if results is not None else np.zeros((258,), np.float32)
                np.save(os.path.join(out_dir, f"{saved}.npy"), keypoints)
                saved += 1
            except Exception as e:
                # Skip this frame if MediaPipe fails
                pass
        
        current += 1
    
    cap.release()
    
    if saved != SEQUENCE_LENGTH:
        # Cleanup incomplete sequence
        for fpath in glob.glob(os.path.join(out_dir, "*.npy")):
            os.remove(fpath)
        try:
            os.rmdir(out_dir)
        except:
            pass
        return False, f"incomplete_{saved}/{SEQUENCE_LENGTH}"
    
    return True, None

# Process videos
built = 0
skipped = 0

with mp_holistic.Holistic(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5,
    static_image_mode=False  # Faster for video
) as holistic:
    with tqdm(total=total_instances, desc="Building MP_Data", unit="inst") as pbar:
        for gloss, insts in by_gloss.items():
            gloss_dir = os.path.join(DATA_PATH, gloss)
            os.makedirs(gloss_dir, exist_ok=True)
            
            # Find next available seq_id
            existing_seqs = [int(d) for d in os.listdir(gloss_dir) 
                           if os.path.isdir(os.path.join(gloss_dir, d)) and d.isdigit()]
            seq_id = max(existing_seqs) + 1 if existing_seqs else 0
            
            for inst in insts:
                success, reason = process_video(inst, gloss, seq_id, holistic)
                
                if success:
                    seq_id += 1
                    built += 1
                else:
                    skip_reasons[reason] += 1
                    skipped += 1
                
                pbar.update(1)
                pbar.set_postfix(gloss=gloss, built=built, skipped=skipped)

print("\n✅ Built sequences:", built)
print("⚠️ Skipped:", skipped)
print("\n📊 Skip reasons breakdown:")
for reason, count in sorted(skip_reasons.items(), key=lambda x: x[1], reverse=True):
    print(f"  {reason}: {count}")
print(f"\nSuccess rate: {built/(built+skipped)*100:.1f}%")
print("MP_Data at:", DATA_PATH)



In [ ]:
import os
from glob import glob

DATA_PATH = "/kaggle/working/sign_language_detection/MP_Data"

def count_sequences(gloss_dir):
    # counts subfolders that have at least 1 npy
    seq_dirs = [d for d in os.listdir(gloss_dir) if os.path.isdir(os.path.join(gloss_dir, d))]
    good = 0
    for sd in seq_dirs:
        npys = glob(os.path.join(gloss_dir, sd, "*.npy"))
        if len(npys) > 0:
            good += 1
    return good

counts = {}
for gloss in sorted(os.listdir(DATA_PATH)):
    gd = os.path.join(DATA_PATH, gloss)
    if os.path.isdir(gd):
        counts[gloss] = count_sequences(gd)

print("Counts per class:")
for g,c in sorted(counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{g:20s} {c}")


In [ ]:
MIN_SEQS = 0

keep_classes = [g for g,c in counts.items() if c >= MIN_SEQS]
drop_classes = [g for g,c in counts.items() if c < MIN_SEQS]

print("Keeping:", keep_classes)
print("Dropping:", drop_classes)


In [ ]:
import numpy as np, os
from glob import glob

DATA_PATH = "/kaggle/working/sign_language_detection/MP_Data"
actions = keep_classes
actions = sorted(actions) 
X, y = [], []
for label, action in enumerate(actions):
    seq_dirs = sorted(glob(os.path.join(DATA_PATH, action, "*")))
    for sdir in seq_dirs:
        seq = []
        ok = True
        for f in range(30):
            fp = os.path.join(sdir, f"{f}.npy")
            if not os.path.exists(fp):
                ok = False
                break
            seq.append(np.load(fp))
        if ok:
            X.append(np.stack(seq))      # (30,258)
            y.append(label)

X = np.array(X, dtype=np.float32)         # (N,30,258)
y = np.array(y, dtype=np.int32)

print("X:", X.shape, "y:", y.shape)


In [ ]:
from collections import defaultdict
import numpy as np

per_class = defaultdict(list)
for i, label in enumerate(y):
    per_class[int(label)].append(i)

min_n = min(len(v) for v in per_class.values())  # should be 8
print("Balancing to", min_n, "samples per class")

keep_idx = []
for label, idxs in per_class.items():
    keep_idx.extend(idxs[:min_n])

keep_idx = np.array(keep_idx)
X = X[keep_idx]
y = y[keep_idx]

print("Balanced X:", X.shape)
print("Balanced y:", y.shape)


In [ ]:
import tensorflow as tf

num_classes = len(actions)

norm = tf.keras.layers.Normalization(axis=-1)
norm.adapt(X)

model = tf.keras.Sequential([
    tf.keras.layers.Input((30, 258)),
    # Ensure norm is adapted correctly or replace with a Rescaling(1./1.) layer
    
    tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(128, return_sequences=True) # Increased units
    ),
    tf.keras.layers.BatchNormalization(), # Helps stabilize training
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(64) # Second layer also bidirectional
    ),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

model.summary()


In [ ]:
import numpy as np
from collections import defaultdict

def stratified_split(X, y, val_frac=0.25, seed=42):
    rng = np.random.default_rng(seed)
    idx_by_class = defaultdict(list)

    for i, label in enumerate(y):
        idx_by_class[int(label)].append(i)

    train_idx, val_idx = [], []

    for label, idxs in idx_by_class.items():
        idxs = np.array(idxs)
        rng.shuffle(idxs)
        n_val = int(len(idxs) * val_frac)
        val_idx.extend(idxs[:n_val])
        train_idx.extend(idxs[n_val:])

    return (
        X[train_idx], X[val_idx],
        y[train_idx], y[val_idx]
    )

X_train, X_val, y_train, y_val = stratified_split(X, y)


callbacks = [
    tf.keras.callbacks.EarlyStopping(
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        patience=4,
        factor=0.5,
        min_lr=1e-5
    ),
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=8,
    callbacks=callbacks,
)


In [ ]:
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print("val_loss:", val_loss, "val_acc:", val_acc)


In [ ]:
import os, random, numpy as np
from glob import glob

def load_saved_sequence_safe(seq_dir, seq_len=30):
    frames = []
    for i in range(seq_len):
        fp = os.path.join(seq_dir, f"{i}.npy")
        if not os.path.exists(fp):
            return None   # sequence is incomplete
        frames.append(np.load(fp).astype(np.float32))
    return np.stack(frames, axis=0)  # (30,258)


In [ ]:
import os, random, numpy as np
from glob import glob

DATA_PATH = "/kaggle/working/sign_language_detection/MP_Data"
SEQ_LEN = 30

def pick_random_seq(action):
    seq_dirs = [
        d for d in glob(os.path.join(DATA_PATH, action, "*"))
        if os.path.isdir(d)
    ]
    random.shuffle(seq_dirs)
    return seq_dirs

random.seed(1)
n = 20
correct = 0
tested = 0

for _ in range(n):
    true_action = random.choice(actions)
    seq_dirs = pick_random_seq(true_action)

    x = None
    sd = None
    for candidate in seq_dirs:
        x = load_saved_sequence_safe(candidate, SEQ_LEN)
        if x is not None:
            sd = candidate
            break

    if x is None:
        print(f"⚠️ no valid sequence for class {true_action}")
        continue

    probs = model.predict(x[None, ...], verbose=0)[0]
    pred = actions[int(np.argmax(probs))]
    conf = float(np.max(probs))

    tested += 1
    correct += int(pred == true_action)
    print(f"true={true_action:12s} pred={pred:12s} conf={conf:.3f} dir={os.path.basename(sd)}")

print(f"\nAccuracy on disk samples: {correct}/{tested} = {correct/max(1,tested):.3f}")


In [ ]:
import json

MODEL_PATH = "/kaggle/working/wlasl_demo.keras"
ACTIONS_PATH = "/kaggle/working/actions.json"

model.save(MODEL_PATH)

with open(ACTIONS_PATH, "w") as f:
    json.dump(actions, f)

print("Saved model to:", MODEL_PATH)
print("Saved actions to:", ACTIONS_PATH)


Code Block 10A

In [ ]:
# Use the trained model to make predictions on the test data (X_test)
# The model outputs the predicted probabilities for each class in the test set
# y_pred will contain the predicted probabilities or class labels for each sample in X_test
y_pred = model.predict(X_test)

# Convert the one-hot encoded true labels (y_test) into class labels (indices)
# np.argmax(y_test, axis=1) finds the index of the maximum value along the second axis (for each sample)
# This index corresponds to the true class label (e.g., [1, 0, 0] -> class 0, [0, 1, 0] -> class 1)
y_true = np.argmax(y_test, axis=1).tolist()

# Convert the predicted probabilities (y_pred) into predicted class labels (indices)
# np.argmax(y_pred, axis=1) finds the index of the highest predicted probability for each sample
# This index corresponds to the predicted class label
y_pred = np.argmax(y_pred, axis=1).tolist()

Code Block 10B

In [ ]:
# Calculate multilabel confusion matrix
conf_matrix = multilabel_confusion_matrix(y_true, y_pred)

for idx, matrix in enumerate(conf_matrix):
    print(f"\nConfusion Matrix for class '{actions[idx]}':")
    df = pd.DataFrame(matrix,
                      index=["Actual Negative", "Actual Positive"],
                      columns=["Predicted Negative", "Predicted Positive"])
    print(df)

Code Block 10C

In [ ]:
# Calculate the accuracy score to evaluate the model's performance
# The accuracy score measures the ratio of correctly predicted labels to the total number of samples
# It compares the true labels (y_true) with the predicted labels (y_pred) and returns the percentage of correct predictions
accuracy_score(y_true, y_pred)

### Note on Kaggle
Kaggle notebooks typically **cannot access your webcam**. Use Section 11 with an **uploaded `.mp4`** (as a Kaggle dataset or placed in `/kaggle/working/test_videos`).


# 11. Testing with Videos

Code Block 11A